# U1 - Final Project: Analyzing Gas Prices in Brazil over 2004 to 2021, comparing with 4 presidents

In [ ]:
import pandas as pd
import pytest
import logging
import matplotlib.pyplot as plt
import matplotlib.style as style

data = pd.read_csv('gas_2004-2021.tsv', sep='\t', lineterminator='\n')
config_logging('./results2.log')

In [ ]:
data.head()
data.info()

## Functions

In [ ]:
def config_logging(path):
    """
    Configures the logging instance
    Args: 
    path: str
    Returns:
    None
    """
    logging.basicConfig(
        filename=path,
        level=logging.INFO,
        filemode='w',
        format='[%(asctime)s] %(name)s - %(levelname)s - %(message)s',
        datefmt='%Y-%m-%d %H:%M:%S'
    )
    

def line_plot_gas_over_time(dataset, label, color):
    """
    Receives a dataframe and plots a line graph
    Args:
    dataset: DataFrame
    label: str
    color: str
    
    Returns:
    None
    """
    try:
        assert(isinstance(label, str))
    except AssertionError:
        logging.error("ERROR: Label must be a string.")
        
    plt.plot(dataset['final_date'], 
         dataset['resell_mean'].rolling(rolling_mean).mean(), 
         label=label, 
         color=color,
         linewidth=2)

def clean_dataset(dataset):
    """
    Receives a DataFrame and cleans it for the problem
    Args:
    dataset: DataFrame
    Returns:
    cleaned_data: DataFrame
    """
    cleaned_data = dataset.copy()
    try:
        cleaned_data.columns = ["initial_date", "final_date", "region", 
                    "state", "product", "gas_stations_number", 
                    "measurement_unit", "resell_mean", "resell_std", 
                    "resell_min_price", "resell_max_price", "resell_mean_margin", 
                    "resell_variant_coef", "mean_price_dist", "std_dist",
                    "min_price_dist", "max_price_dist", "variation_dist_coef"]
    
        cleaned_data['final_date'] = pd.to_datetime(cleaned_data['final_date'])
        cleaned_data.drop("initial_date", axis=1, inplace=True)
    except KeyError:
        logging.error("ERROR: final_date or initial_date not found on this DataFrame")
    except ValueError:
        logging.error("ERROR: There are less columns than the specified.")

    #Removing initial_date, since it won't be used
    cleaned_data.sort_values('final_date', inplace=True)
    cleaned_data.reset_index(drop=True, inplace=True)
    
    #Removing accents
    cleaned_data['product'].replace({'ÓLEO DIESEL': 'OLEO DIESEL'}, inplace=True)
    cleaned_data['product'].replace({'ÓLEO DIESEL S10': 'OLEO DIESEL S10'}, inplace=True)
    
    logging.info("SUCCESS: dataframe was cleaned successfully.")
    
    return cleaned_data
    

## Data Cleaning

In the line above, we could see that the columns are hard to work with, so let's rename them.

In [ ]:
data = clean_dataset(data)

## Telling a Story

Based on this data, there are a few questions we can ask and answer in this project.
* How did Gasoline, Ethanol and Diesel evolved throughout the years?
* How did Gasoline evolve over the years in each Brazilian region?
* How did Gasoline evolve in each of the four president's mandates? (Lula, Dilma, Temer and Bolsonaro)
* How did GLP (Home cooking Gas) evolve in each mandate? (Lula, Dilma, Temer and Bolsonaro)

In [ ]:
#Question 1
data_ethanol = data[data['product'] == "ETANOL HIDRATADO"]
data_gasoline = data[data['product'] == "GASOLINA COMUM"]
data_additive_gasoline = data[data['product'] == "GASOLINA ADITIVADA"]
data_diesel = data[data['product'] == "OLEO DIESEL"]
data_diesel_s10 = data[data['product'] == "OLEO DIESEL S10"]
data_glp = data[data['product'] == "GLP"]

#Question 2
gasoline_centro_oeste = data_gasoline[data_gasoline['region'] == 'CENTRO OESTE']
gasoline_nordeste = data_gasoline[data_gasoline['region'] == 'NORDESTE']
gasoline_sul = data_gasoline[data_gasoline['region'] == 'SUL']
gasoline_sudeste = data_gasoline[data_gasoline['region'] == 'SUDESTE']
gasoline_norte = data_gasoline[data_gasoline['region'] == 'NORTE']

#Question 3
gasoline_lula = data_gasoline.copy()[(data_gasoline['final_date'].dt.year >= 2004) & (data_gasoline['final_date'].dt.year <= 2010)]
gasoline_dilma = data_gasoline.copy()[(data_gasoline['final_date'].dt.year >= 2010) & (data_gasoline['final_date'].dt.year <= 2016)]
gasoline_temer = data_gasoline.copy()[(data_gasoline['final_date'].dt.year >= 2016) & (data_gasoline['final_date'].dt.year <= 2019)]
gasoline_bolsonaro = data_gasoline.copy()[data_gasoline['final_date'].dt.year >= 2019]

#Question 4
glp_lula = data_glp.copy()[(data_glp['final_date'].dt.year >= 2004) & (data_glp['final_date'].dt.year <= 2010)]
glp_dilma = data_glp.copy()[(data_glp['final_date'].dt.year >= 2010) & (data_glp['final_date'].dt.year <= 2016)]
glp_temer = data_glp.copy()[(data_glp['final_date'].dt.year >= 2016) & (data_glp['final_date'].dt.year <= 2019)]
glp_bolsonaro = data_glp.copy()[data_glp['final_date'].dt.year >= 2019]

In [ ]:
rolling_mean = 200

### 1.0 Gasoline, Ethanol and Diesel evolution through the years

In [ ]:
#Using a 200-day rolling_mean
plt.figure(figsize=(12,6))

line_plot_gas_over_time(data_gasoline, 'Gasoline', 'c')
line_plot_gas_over_time(data_diesel, 'Diesel', 'm')
line_plot_gas_over_time(data_ethanol, 'Ethanol', 'g')

### Highlihting the peak of the crisis
plt.axvspan(xmin=18440, xmax=18800, ymin=0.009,
           alpha=0.3, color='black')

#Adding titles
plt.suptitle("Gasoline, Diesel and Ethanol evolution in Brazil from 2004 to 2021", weight='bold')
plt.title("Fuel prices have reached their maximum historical value during pandemics", fontsize='small')

#plt.xlabel('Year')
#plt.ylabel('Mean Price (R$/l)')

#Adding footer
plt.annotate('©DATAQUEST' + ' '*20 + 'Source: National Agency of Petroleum, Natural Gas and Bio fuels (ANP in Portuguese)', (0,0), (-20, -50), xycoords='axes fraction', textcoords='offset points', va='top', backgroundcolor = '#4d4d4d', color = '#f0f0f0')

#Increasing grid opacity
plt.grid(alpha=0.5)

plt.legend()
plt.show()

### 2.0 Gasoline evolution in every brazilian region

In [ ]:
plt.figure(figsize=(12,6))

line_plot_gas_over_time(gasoline_centro_oeste, 'Centro-Oeste', 'orange')
line_plot_gas_over_time(gasoline_nordeste, 'Nordeste', 'g')
line_plot_gas_over_time(gasoline_norte, 'Norte', 'b')
line_plot_gas_over_time(gasoline_sul, 'Sul', 'r')
line_plot_gas_over_time(gasoline_sudeste, 'Sudeste', 'aquamarine')

#Adding titles
plt.suptitle("Gasoline evolution in Brazil from 2004 to 2021", weight='bold')
plt.title("Gasoline evolution separated by region", fontsize='small')

#plt.xlabel('Year')
#plt.ylabel('Mean Price (R$/l)')

#Adding footer
plt.annotate('©DATAQUEST' + ' '*20 + 'Source: National Agency of Petroleum, Natural Gas and Bio fuels (ANP in Portuguese)', (0,0), (-20, -50), xycoords='axes fraction', textcoords='offset points', va='top', backgroundcolor = '#4d4d4d', color = '#f0f0f0')

#Increasing grid opacity
plt.grid(alpha=0.5)

plt.legend()
plt.show()

### 3.0 Gasoline in every President Mandate from 2004 to 2022

In [ ]:
plt.figure(figsize=(12,6))

line_plot_gas_over_time(gasoline_lula, "Lula", 'r')
line_plot_gas_over_time(gasoline_dilma, "Dilma", 'orange')
line_plot_gas_over_time(gasoline_temer, "Temer", 'blue')
line_plot_gas_over_time(gasoline_bolsonaro, "Bolsonaro", 'green')

#Adding titles
plt.suptitle("Gasoline in every Government from 2004 to 2022", weight='bold')
plt.title("Prices in every president Mandate (Lula, Dilma, Temer and Bolsonaro)", fontsize='small')

#plt.xlabel('Year')
#plt.ylabel('Mean Price (R$/l)')

#Adding footer
plt.annotate('©DATAQUEST' + ' '*20 + 'Source: National Agency of Petroleum, Natural Gas and Bio fuels (ANP in Portuguese)', (0,0), (-20, -50), xycoords='axes fraction', textcoords='offset points', va='top', backgroundcolor = '#4d4d4d', color = '#f0f0f0')

#Increasing grid opacity
plt.grid(alpha=0.5)

plt.legend()
plt.show()

### 4.0 GLP (Home Cooking Gas) Prices for every President

In [ ]:
plt.figure(figsize=(12,6))

line_plot_gas_over_time(glp_lula, "Lula", 'r')
line_plot_gas_over_time(glp_dilma, "Dilma", 'orange')
line_plot_gas_over_time(glp_temer, "Temer", 'blue')
line_plot_gas_over_time(glp_bolsonaro, "Bolsonaro", 'green')

#Adding titles
plt.suptitle("GLP Price in every Government from 2004 to 2022", weight='bold')
plt.title("Prices in every president Mandate (Lula, Dilma, Temer and Bolsonaro)", fontsize='small')

#plt.xlabel('Year')
#plt.ylabel('Mean Price (R$/l)')

#Adding footer
plt.annotate('©DATAQUEST' + ' '*20 + 'Source: National Agency of Petroleum, Natural Gas and Bio fuels (ANP in Portuguese)', (0,0), (-20, -50), xycoords='axes fraction', textcoords='offset points', va='top', backgroundcolor = '#4d4d4d', color = '#f0f0f0')

#Increasing grid opacity
plt.grid(alpha=0.5)

plt.legend()
plt.show()